In [48]:
from ipyparallel import Client
import numpy as np

c = Client(profile='mpi')
with c[:].sync_imports():
    import numpy as np

print(c.ids)
    
#dview = c[:]
dview = c.direct_view()

def square(x):
    return x**2
 
result = dview.map_sync(square, range(16))
print(result)


importing numpy on engine(s)
[0, 1, 2, 3]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225]


In [32]:
def multi(a, b):
    return a*b
 
results = dview.map_sync(multi, [0, 1, 3, 5], [2, 4, 6, 8])
print(results)


[0, 4, 18, 40]


In [35]:
import random
with dview.sync_imports():
    import random
    
def mc_pi(n):
    count = 0
    for i in range(n):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) <= 1:
            count += 1
    return count/n


est_pi = mc_pi(10**6)
print(est_pi*4) 

est_pis = dview.map_sync(mc_pi, [250000]*4)
print(sum(est_pis)*4/4)


importing random on engine(s)
3.141228
3.14094


In [50]:

from ipyparallel import Client
c = Client(profile='mpi')

view = c[:]

view.activate() # enable magics

# run the contents of the file on each engine:
view.run('other/psum.py')

view.scatter('a',np.arange(16,dtype='float'))

print(view['a'])

%px totalsum = psum(a)

view['totalsum']


[array([ 0.,  1.,  2.,  3.]), array([ 4.,  5.,  6.,  7.]), array([  8.,   9.,  10.,  11.]), array([ 12.,  13.,  14.,  15.])]


[ipyparallel.error.TaskAborted('43bd0863-ad50-458f-9f01-7cef1af342bb'),
 ipyparallel.error.TaskAborted('6e1e8737-6ea7-4dec-8c41-18e516450130'),
 ipyparallel.error.TaskAborted('a4067801-ab14-4c5e-940f-ba6f0cb69a61'),
 ipyparallel.error.TaskAborted('df3e843d-be41-4cb9-84ba-6cf5241071d4')]

In [46]:
from mpi4py import MPI

#MPI_Init()

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
print('np: ', comm.Get_size())
print(MPI.Is_initialized())

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1, tag=11)
elif rank == 1:
    data = comm.recv(source=0, tag=11)



np:  1
True


Exception: MPI_ERR_RANK: invalid rank